In [1]:
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np

In [40]:
cfdata = pd.read_csv('./data/cfdata.csv') #전체 카페 데이터

In [41]:
date = pd.to_datetime(cfdata['DCBYMD'],format='%Y-%m-%d')
opdate = pd.to_datetime(cfdata['APVPERMYMD'],format='%Y-%m-%d')
cfdata['DCBYMD'] = date
cfdata['APVPERMYMD'] = opdate
cfdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24541 entries, 0 to 24540
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   24541 non-null  int64         
 1   APVPERMYMD   24541 non-null  datetime64[ns]
 2   TRDSTATEGBN  24541 non-null  int64         
 3   TRDSTATENM   24541 non-null  object        
 4   DCBYMD       8624 non-null   datetime64[ns]
 5   SITEAREA     23486 non-null  object        
 6   RDNWHLADDR   24541 non-null  object        
 7   BPLCNM       24541 non-null  object        
 8   X            24541 non-null  float64       
 9   Y            24541 non-null  float64       
 10  region       24541 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(5)
memory usage: 2.1+ MB


In [42]:
#cfdata = cfdata.drop(['Unnamed: 0'],axis = 1)

In [43]:
cfdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,X,Y,region
0,2013-11-13,3,폐업,2017-10-11,22.53,서울특별시 강동구 양재대로87길 11 (성내동),탭플레이 커피,127.134306,37.525438,강동구
1,2015-07-08,3,폐업,2017-07-10,22.00,"서울특별시 강동구 진황도로27길 23, 1층 (천호동)",아카페,127.131877,37.539416,강동구
2,2014-03-20,3,폐업,2017-07-17,74.21,"서울특별시 강동구 구천면로 428 (명일동, 명일센트로빌 205호)",CAFE TENUTO 테누토,127.146158,37.550130,강동구
3,2014-12-05,3,폐업,2017-06-02,337.19,"서울특별시 강동구 강동대로 205, 1층 (성내동)",할리스커피 올림픽공원북문점,127.129210,37.524394,강동구
4,2013-01-03,3,폐업,2017-05-18,30.00,서울특별시 강동구 풍성로63길 84 (둔촌동),도우드림,127.140339,37.531725,강동구
...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,"서울특별시 서초구 강남대로39길 15-7 (서초동,브니엘빌딩 1층)",카페 엘리,127.032416,37.485785,서초구
24537,2010-09-16,1,영업/정상,NaT,50.00,서울특별시 서초구 바우뫼로41길 8 (양재동),카페레즈,127.041496,37.481075,서초구
24538,2010-05-27,1,영업/정상,NaT,142.50,"서울특별시 서초구 사평대로 106, 1층 (반포동)",마켓 비노플라워,126.996156,37.498874,서초구
24539,2009-10-29,1,영업/정상,NaT,62.00,서울특별시 서초구 바우뫼로41길 88 (양재동),까페원더,127.039916,37.484342,서초구


In [6]:
x=cfdata['X'].reset_index(drop=True)
y=cfdata['Y'].reset_index(drop=True)

x=x.astype('str')
y=y.astype('str')

In [63]:
url = 'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=SC4&radius=1000&sort=distance&x='+ x[24537] + '&y=' + y[24537]
headers = {"Authorization":"KakaoAK 6341d1e76477fbac498a469adc87108d"}
result = json.loads(str(requests.get(url, headers=headers).text))
aa = result['documents']    #[i] 카페 주변의 모든 학교(초,중,고,대)
#aa

In [48]:
num_unv = []
unv_name=[]
unv_dist=[]

for j in range(len(x)):

    url = 'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=SC4&radius=1000&sort=distance&x='+ x[j] + '&y=' + y[j]
    headers = {"Authorization":"KakaoAK 6341d1e76477fbac498a469adc87108d"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    aa = result['documents']    #[i] 카페 주변의 모든 학교(초,중,고,대)
    
    unv=[]
    dist=[]
    #모든 학교 중 대학교 찾기
    for i in range(len(aa)):
        if aa[i]['category_name'] == '교육,학문 > 학교 > 대학교' :
            unv.append(aa[i]['place_name'])   #대학교 이름
            dist.append(aa[i]['distance'])
            
    #주변에 대학교가 있을 시 대학 이름,거리 리스트 추가
    if len(unv) > 0 :
        unv_name.append(unv[0])
        unv_dist.append(dist[0])
    else : #없을 시 '없음' 추가
        unv_name.append('없음')
        unv_dist.append('x')
        
        
    num_unv.append(len(unv))    #카페 주변의 대학교 갯수
    
    
#print(unv_name)
#print(num_unv)
len(num_unv)

24541

In [49]:
num_unv[:20]

[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [50]:
unv_name[2000:2020]

['없음',
 '서울과학기술대학교',
 '없음',
 '없음',
 '없음',
 '없음',
 '서울과학기술대학교',
 '서울과학기술대학교',
 '광운대학교',
 '광운대학교',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음',
 '없음']

In [51]:
unv_dist[2000:2020]

['x',
 '878',
 'x',
 'x',
 'x',
 'x',
 '794',
 '940',
 '634',
 '165',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x']

In [52]:
cfdata['num_unv'] = num_unv
cfdata['unv_name'] = unv_name
cfdata['unv_dist'] = unv_dist

In [53]:
cfdata

,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,X,Y,region,num_unv,unv_name,unv_dist
0,2013-11-13,3,폐업,2017-10-11,22.53,서울특별시 강동구 양재대로87길 11 (성내동),탭플레이 커피,127.134306,37.525438,강동구,1,한국체육대학교,662
1,2015-07-08,3,폐업,2017-07-10,22.00,"서울특별시 강동구 진황도로27길 23, 1층 (천호동)",아카페,127.131877,37.539416,강동구,0,없음,x
2,2014-03-20,3,폐업,2017-07-17,74.21,"서울특별시 강동구 구천면로 428 (명일동, 명일센트로빌 205호)",CAFE TENUTO 테누토,127.146158,37.550130,강동구,0,없음,x
3,2014-12-05,3,폐업,2017-06-02,337.19,"서울특별시 강동구 강동대로 205, 1층 (성내동)",할리스커피 올림픽공원북문점,127.129210,37.524394,강동구,1,한국체육대학교,490
4,2013-01-03,3,폐업,2017-05-18,30.00,서울특별시 강동구 풍성로63길 84 (둔촌동),도우드림,127.140339,37.531725,강동구,0,없음,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,2011-03-02,1,영업/정상,NaT,98.10,"서울특별시 서초구 강남대로39길 15-7 (서초동,브니엘빌딩 1층)",카페 엘리,127.032416,37.485785,서초구,2,횃불트리니티 신학대학원대학교,746
24537,2010-09-16,1,영업/정상,NaT,50.00,서울특별시 서초구 바우뫼로41길 8 (양재동),카페레즈,127.041496,37.481075,서초구,3,서울외국어대학원대학교,235
24538,2010-05-27,1,영업/정상,NaT,142.50,"서울특별시 서초구 사평대로 106, 1층 (반포동)",마켓 비노플라워,126.996156,37.498874,서초구,1,가톨릭대학교 성의교정,839
24539,2009-10-29,1,영업/정상,NaT,62.00,서울특별시 서초구 바우뫼로41길 88 (양재동),까페원더,127.039916,37.484342,서초구,3,서울외국어대학원대학교,341


In [54]:
cfdata.to_csv('./data/cfdata_sc4.csv',encoding='utf-8')

In [55]:
cf_sc4 = pd.read_csv('./data/cfdata_sc4.csv')

In [57]:
dj_sc4 = cf_sc4[cf_sc4['region']=='동작구']

In [58]:
dj_sc4

,Unnamed: 0,APVPERMYMD,TRDSTATEGBN,TRDSTATENM,DCBYMD,SITEAREA,RDNWHLADDR,BPLCNM,X,Y,region,num_unv,unv_name,unv_dist
572,572,2009-03-23,3,폐업,2017-10-19,25.10,서울특별시 동작구 여의대방로22나길 1 (신대방동),커피나무,126.920799,37.497518,동작구,0,없음,x
573,573,2015-08-06,3,폐업,2017-05-16,154.62,"서울특별시 동작구 상도로 381, 지상1,2층 (상도동)",카페아이엠티숭실대점,126.954995,37.495140,동작구,2,숭실대학교,262
574,574,2015-08-10,3,폐업,2017-07-19,20.00,"서울특별시 동작구 만양로14가길 30-3, 지상1층 (노량진동)",그리다꿈노량진점,126.944998,37.512912,동작구,0,없음,x
575,575,2001-10-25,3,폐업,2017-10-17,19.16,서울특별시 동작구 흑석로 12 (상도1동),늘솜카페,126.952048,37.504768,동작구,1,중앙대학교 서울캠퍼스,265
576,576,2013-10-21,3,폐업,2017-09-25,25.00,"서울특별시 동작구 국사봉길 60 (상도동, 1층)",뽐뽐,126.930340,37.497146,동작구,0,없음,x
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22760,22760,2012-02-17,1,영업/정상,NaN,16.20,"서울특별시 동작구 노량진로 162 (노량진동, 1층)",카페1987,126.943922,37.513387,동작구,0,없음,x
22840,22840,2012-05-16,1,영업/정상,NaN,202.00,"서울특별시 동작구 노량진로 144 (노량진동, 외 2필지, 2층)",설빙 노량진점,126.941829,37.513423,동작구,0,없음,x
22841,22841,2012-05-21,1,영업/정상,NaN,18.00,"서울특별시 동작구 상도로 161 (상도동, 1층)",미모,126.936854,37.503770,동작구,0,없음,x
22842,22842,2010-10-07,1,영업/정상,NaN,5.17,"서울특별시 동작구 노량진로 162 (노량진동,1층 8-4호)",커피자리,126.943922,37.513387,동작구,0,없음,x
